In [1]:
from utils import marich, dataset
from nets import *
from tqdm import tqdm
import numpy as np
import torch
import pickle
from torchvision import transforms,models
from sklearn.model_selection import train_test_split

Files already downloaded and verified


In [2]:
import pickle
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo)
    return dict

In [3]:
from torchvision import transforms
from torch.utils.data import Dataset
imagenet = unpickle("val_data") # Valdata to be downloaded from ImageNet website. It is the validation split of ImageNet 32, containing 50k samples
imagenet = torch.tensor(np.float32(imagenet['data'].reshape(50000,3,32,32)/255))
train_transform = transforms.Compose([
transforms.Normalize([0.473, 0.450, 0.401], [0.258, 0.251, 0.265])
])

In [4]:
class dataset(Dataset):
    def __init__(self, X, transform = None):
        self.X = X
        # self.Y = Y
        self.transform = transform

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        if self.transform:
            x = self.transform(self.X[idx])
        else:
            x = self.X[idx]
        # y = self.Y[idx]
        return x

In [5]:
img_data = dataset(imagenet, transform = train_transform)

In [6]:
# imagenet_data = torch.stack([train_transform(x) for x in imagenet])

In [7]:
# torch.save(imagenet_data, "imagenet_resnet.pt")

In [8]:
# torch.save(img_data, 'img_data.pt')

In [9]:
# img_data = torch.load("img_data.pt")

In [10]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        base = models.resnet18(pretrained=True)
        self.base = nn.Sequential(*list(base.children())[:-1])
        in_features = base.fc.in_features
        self.drop = nn.Dropout()
        self.final = nn.Linear(in_features,10)
    
    def forward(self,x): 
        x = self.base(x)
        x = self.drop(x.view(-1,self.final.in_features))
        x = self.final(x)
        x = F.softmax(x, dim = 1)
        return x
    
model = Model().cuda()
[x for x,y in model.named_children()]

c:\ProgramData\Anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
c:\ProgramData\Anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


['base', 'drop', 'final']

In [11]:
# model.load_state_dict(torch.load("new_resnet18_cifar.pt"))

In [12]:
# model = ResNet().cuda()
# model.load_state_dict(torch.load("cnn.pt"))

In [13]:
def test_accuracy(net, testloader):
    correct = 0
    with torch.no_grad():
        net.eval()
        for images, labels in testloader:
            images, labels = images.to(device), labels.to(device)
            outputs = net(images)
            predicted = torch.max(outputs.data, 1)[1]

            correct += (predicted == labels).sum().item()
    
    return correct / len(testloader.dataset)

In [14]:
# # unlab_x = []
# unlab_y = []
# for j in tqdm(img_data):
#     model.eval()
#     j = j.to(device)
#     # unlab_x.append(j)
#     unlab_y.append(torch.argmax(model(j.unsqueeze(dim = 0))).item())

In [15]:
# # with open('./imagenet_target/imagnet_resnet_x.pkl', 'wb') as f:
# #   pickle.dump(unlab_x, f)

# with open('./imagenet_target/imagnet_resnet_small_y.pkl', 'wb') as f:
#   pickle.dump(unlab_y, f)

In [16]:
# with open('./imagenet_target/imagnet_resnet_x.pkl', 'rb') as f:
#   unlab_x = pickle.load(f)

with open('./imagenet_target/imagnet_resnet_small_y.pkl', 'rb') as f:
  unlab_y = pickle.load(f)

In [17]:
class basic_dataset(Dataset):
    def __init__(self, X, Y = None, transform = None):
        self.X = X
        self.Y = Y
        self.transform = transform

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        if self.transform:
            x = self.transform(self.X[idx])
        else:
            x = self.X[idx]
        if self.Y != None:
            y = self.Y[idx]
            return x, y
        else:
            return x

class dataset2(Dataset):
    def __init__(self, X, Y = None, transform = None):
        self.X = X
        self.Y = Y
        self.transform = transform

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        if self.transform:
            x = self.transform(self.X[idx])
        else:
            x = self.X[idx]
        if self.Y != None:
            y = self.Y[idx]
            return x, y
        else:
            return x
    def get_data(self,idx):
        if self.transform:
            x = self.transform(self.X[idx])
        else:
            x = self.X[idx]
        return x
    def get_dataset(self, idx):
        x = self.X[idx]
        y = torch.tensor(self.Y)[idx]
        return basic_dataset(x,y)
    def get_label(self,idx):
        y = torch.tensor(self.Y)[idx]
        return y
    def get_data_label_loader(self,idx):
        if self.transform:
            x = self.transform(self.X[idx])
        else:
            x = self.X[idx]
        y = torch.tensor(self.Y)[idx]
        
        return_set = basic_dataset(x,y)
        return torch.utils.data.DataLoader(return_set, batch_size = 64)

In [18]:
train_id, val_id = train_test_split(range(50000), test_size = 0.2)

In [19]:
unlab_dataset_train = dataset2(img_data[train_id], torch.tensor(unlab_y)[train_id])
unlab_dataset_val = dataset2(img_data[val_id], torch.tensor(unlab_y)[val_id])

In [20]:
cifar10 = datasets.CIFAR10('./cifar10/train/', download=True, train=False, transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))
    ]))
testloader = DataLoader(cifar10, batch_size = 64)
validloader = DataLoader(unlab_dataset_val, batch_size = 64)

Files already downloaded and verified


In [21]:
# cifar10 = datasets.CIFAR10('./cifar10/train/', download=True, train=False, transform = transforms.Compose([
#     # transforms.Resize(224),
#     transforms.ToTensor(),
#     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))
#     ]))
# testloader = DataLoader(cifar10, batch_size = 512)
# validloader = DataLoader(unlab_dataset_val, batch_size = 512)

In [22]:
class ResNet(nn.Module):
    def __init__(self):
        super(ResNet, self).__init__()
        
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2)
        )

        self.res1 = nn.Sequential(nn.Sequential(
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True)
        ), nn.Sequential(
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True))
        )
        
        self.conv3 = nn.Sequential(
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2)
        )
        self.conv4 = nn.Sequential(
            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2)
        )

        self.res2 = nn.Sequential(nn.Sequential(
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True)
        ), nn.Sequential( 
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True))
        )

        self.classifier = nn.Sequential(
            nn.MaxPool2d(4), 
            nn.Flatten(), 
            nn.Linear(512, 10)
        )
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.res1(x) + x
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.res2(x) + x
        x = self.classifier(x)
        
        return x

In [23]:
acc_marich = []

for i in tqdm(range(10)):
  cnn_img_attack = CNN_img()
  tl_cnn_img, vl_cnn_img, tal_cnn_img, samp_cnn_img = marich(cnn_img_attack, unlab_dataset_train, validloader, testloader, budget = 1200, init_points = 500, rounds = 10, epochs = 8, LR = 0.2, gamma1 = 0.8, gamma2 = 0.8, model_name = "resnet_cnn_attack"+str(i)+".pt", sampling = "all_elg", device = "cuda", model_type = "cnn_3c_small")
  
  acc_marich.append(tal_cnn_img)

  np.save("./results/acc_marich_resnet_cnn_img.npy", np.array(acc_marich))
  # np.save("/content/drive/

  0%|          | 0/10 [00:00<?, ?it/s]C:\Users\pkpra\AppData\Local\Temp/ipykernel_23384/108530465.py:48: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(self.Y)[idx]


Using CUDA
Test accuracy:  10.0
Training samples:  500
Epoch:  1


e:\pratik\new_marich\utils.py:44: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels.append(torch.tensor(l))


Train loss:  2.238125205039978
Validation loss:  2.2730389279165086
Epoch time -----  1.392977237701416  sec
validation loss minimum, saving model
Epoch:  2
Train loss:  2.1905978620052338
Validation loss:  2.2519156067234696
Epoch time -----  1.1179296970367432  sec
validation loss minimum, saving model
Epoch:  3
Train loss:  2.1646809577941895
Validation loss:  2.236851620825992
Epoch time -----  1.1718330383300781  sec
validation loss minimum, saving model
Epoch:  4
Train loss:  2.1627182364463806
Validation loss:  2.2212671383171325
Epoch time -----  1.1466987133026123  sec
validation loss minimum, saving model
Epoch:  5
Train loss:  2.160514086484909
Validation loss:  2.2060292192325472
Epoch time -----  1.0821919441223145  sec
validation loss minimum, saving model
Epoch:  6
Train loss:  2.1580896377563477
Validation loss:  2.1947842115049907
Epoch time -----  1.0698001384735107  sec
validation loss minimum, saving model
Epoch:  7
Train loss:  2.1614859998226166
Validation loss:  

C:\Users\pkpra\AppData\Local\Temp/ipykernel_23384/108530465.py:58: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(self.Y)[idx]
e:\pratik\new_marich\utils.py:383: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  probs = F.softmax(probs).detach().cpu().numpy()


...

Using gradient sampling on  1657  points


C:\Users\pkpra\AppData\Local\Temp/ipykernel_23384/108530465.py:48: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(self.Y)[idx]
e:\pratik\new_marich\utils.py:44: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels.append(torch.tensor(l))


Budget:  13179
Training samples:  13179
Training
Epoch:  1
Train loss:  1.583406371399037
Validation loss:  1.749295481450998
Epoch time -----  4.444397211074829  sec
Epoch:  2
Train loss:  1.0606752584860162
Validation loss:  1.5725448693439459
Epoch time -----  4.002802848815918  sec
Epoch:  3
Train loss:  0.8110090019633469
Validation loss:  1.5934539429701058
Epoch time -----  4.067033529281616  sec
Epoch:  4
Train loss:  0.7518638488158439
Validation loss:  1.6004276264245343
Epoch time -----  3.419981002807617  sec
Epoch:  5
Train loss:  0.7525113702399059
Validation loss:  1.6041358789061284
Epoch time -----  3.818267583847046  sec
Epoch:  6
Train loss:  0.7529081728273225
Validation loss:  1.60389397971949
Epoch time -----  4.2278289794921875  sec
Epoch:  7
Train loss:  0.7516017576444496
Validation loss:  1.5998064389654025
Epoch time -----  4.122826099395752  sec
Epoch:  8
Train loss:  0.7503706566917087
Validation loss:  1.6029897578962289
Epoch time -----  3.890368223190307

100%|██████████| 10/10 [1:15:21<00:00, 452.10s/it]

Test accuracy:  57.93


In [ ]:
test_accuracy(cnn_img_attack, testloader)

0.7253

In [ ]:
np.save("./results/samp_resnet_cnn_img.npy", np.array(samp_cnn_img))

In [ ]:
samp_cnn_img

[0, 500, 1257, 2022, 2794, 3577, 4366, 5162, 5967, 6781, 7601, 8429]